<a href="https://colab.research.google.com/github/jaeminSon/Google_Code_Jam/blob/master/Ensemble_with_Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# !wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
# !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
# !unzip train2017.zip
!unzip val2017.zip
# !unzip annotations_trainval2017.zip

In [ ]:
# remove truncated image & zip file in training set
!rm train2017.zip
!rm train2017/000000081166.jpg

In [ ]:
# # install detectron2: (Colab has CUDA 10.1 + torch 1.7)
!pip install pyyaml==5.1
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import torch
assert torch.__version__.startswith("1.7")
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

In [92]:
import json
import random
import os

# # sample training dataset for inference (compute time limit)
# path_sampled_json = "annotations/sampled_instance_train2017.json"
# if not os.path.exists(path_sampled_json):
#   with open("./annotations/instances_train2017.json") as f:
#     label = json.load(f)
#   new_labels = {"annotations":[], "images":[]}
#   for key in ['info', 'licenses', 'categories', 'annotations']:
#     new_labels[key] = label[key]
#   imgs = os.listdir("train2017")
#   selected_imgs = set(random.sample(imgs, len(imgs)//20))
#   for label_img in label["images"]:
#     if label_img["file_name"] in selected_imgs:
#       new_labels["images"].append(label_img)

#   with open(path_sampled_json, "w") as f:
#     json.dump(new_labels, f)

# # register dataset
# register_coco_instances("train2017", {}, path_sampled_json, "./train2017")
# dicts_train2017 = DatasetCatalog.get("train2017")
# metadata_train2017 = MetadataCatalog.get("train2017")
register_coco_instances("val2017", {}, "./annotations/instances_val2017.json", "./val2017")
dicts_val2017 = DatasetCatalog.get("val2017")
metadata_val2017 = MetadataCatalog.get("val2017")

WARNING [01/24 11:42:51 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/24 11:42:51 d2.data.datasets.coco]: Loaded 5000 images in COCO format from ./annotations/instances_val2017.json


In [89]:
dict_cfg = {}
# faster rcnn X101-FPN
dict_cfg["x101"] = get_cfg()
dict_cfg["x101"].merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
dict_cfg["x101"].MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
dict_cfg["x101"].DATASETS.TEST = ("val2017", )
# faster rcnn R101-FPN
dict_cfg["r101_fpn"] = get_cfg()
dict_cfg["r101_fpn"].merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
dict_cfg["r101_fpn"].MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
dict_cfg["r101_fpn"].DATASETS.TEST = ("val2017", )
# RPN & Fast R101-C4
dict_cfg["r101_c4"] = get_cfg()
dict_cfg["r101_c4"].merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml"))
dict_cfg["r101_c4"].MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_C4_3x.yaml")
dict_cfg["r101_c4"].DATASETS.TEST = ("val2017", )
os.makedirs(dict_cfg["x101"].OUTPUT_DIR, exist_ok=True)

dict_models = {}
dict_models["x101"] = DefaultPredictor(dict_cfg["x101"]).model
dict_models["r101_fpn"] = DefaultPredictor(dict_cfg["r101_fpn"]).model
dict_models["r101_c4"] = DefaultPredictor(dict_cfg["r101_c4"]).model


In [ ]:
# run inference models
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# for dataset in ["train2017", "val2017"]:
# for dataset in ["val2017"]:
for dataset in ["train2017"]:
  for model_name, model in dict_models.items():
    print(model_name)
    evaluator = COCOEvaluator(dataset, dict_cfg[model_name], False, output_dir="./{}_{}_outputs/".format(dataset, model_name))
    val_loader = build_detection_test_loader(dict_cfg[model_name], dataset)
    print(inference_on_dataset(model, val_loader, evaluator))

In [ ]:
# !cp -r val2017_x101_outputs gdrive/MyDrive
# !cp -r val2017_r101_fpn_outputs gdrive/MyDrive
# !cp -r val2017_r101_c4_outputs gdrive/MyDrive
!cp -r train2017_x101_outputs gdrive/MyDrive
!cp -r train2017_r101_fpn_outputs gdrive/MyDrive
!cp -r train2017_r101_c4_outputs gdrive/MyDrive

In [93]:
# measure individual model's performance
for dataset in ["val2017"]:
  for model_name, model in dict_models.items():
    evaluator = COCOEvaluator(dataset, dict_cfg[model_name], False)
    predictions = torch.load("./gdrive/MyDrive/{}_{}_outputs/instances_predictions.pth".format(dataset, model_name))
    evaluator._predictions = predictions
    evaluator.evaluate()

WARNING [01/24 11:42:58 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass tasks in directly
[01/24 11:43:03 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[01/24 11:43:03 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.14s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 7.92 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 0.88 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.430
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.637
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.469
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.272
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium 

In [ ]:
dataset = "val2017"
dict_predictions = {}
for model_name in dict_models.keys():
  dict_predictions[model_name] = torch.load("./gdrive/MyDrive/{}_{}_outputs/instances_predictions.pth".format(dataset, model_name))
  dict_predictions[model_name] = sorted(dict_predictions[model_name], key=lambda x:x["image_id"])

In [85]:
# weighted box fusion code from https://github.com/ZFTurbo/Weighted-Boxes-Fusion
import warnings
import numpy as np
from numba import jit


@jit(nopython=True)
def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou


def prefilter_boxes(boxes, scores, labels, weights, thr):
    # Create dict with boxes stored by its label
    new_boxes = dict()

    for t in range(len(boxes)):

        if len(boxes[t]) != len(scores[t]):
            print('Error. Length of boxes arrays not equal to length of scores array: {} != {}'.format(len(boxes[t]), len(scores[t])))
            exit()

        if len(boxes[t]) != len(labels[t]):
            print('Error. Length of boxes arrays not equal to length of labels array: {} != {}'.format(len(boxes[t]), len(labels[t])))
            exit()

        for j in range(len(boxes[t])):
            score = scores[t][j]
            if score < thr:
                continue
            label = int(labels[t][j])
            box_part = boxes[t][j]
            x1 = float(box_part[0])
            y1 = float(box_part[1])
            x2 = float(box_part[2])
            y2 = float(box_part[3])

            # Box data checks
            if x2 < x1:
                warnings.warn('X2 < X1 value in box. Swap them.')
                x1, x2 = x2, x1
            if y2 < y1:
                warnings.warn('Y2 < Y1 value in box. Swap them.')
                y1, y2 = y2, y1
            if x1 < 0:
                warnings.warn('X1 < 0 in box. Set it to 0.')
                x1 = 0
            if x1 > 1:
                warnings.warn('X1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x1 = 1
            if x2 < 0:
                warnings.warn('X2 < 0 in box. Set it to 0.')
                x2 = 0
            if x2 > 1:
                warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                x2 = 1
            if y1 < 0:
                warnings.warn('Y1 < 0 in box. Set it to 0.')
                y1 = 0
            if y1 > 1:
                warnings.warn('Y1 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y1 = 1
            if y2 < 0:
                warnings.warn('Y2 < 0 in box. Set it to 0.')
                y2 = 0
            if y2 > 1:
                warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
                y2 = 1
            if (x2 - x1) * (y2 - y1) == 0.0:
                warnings.warn("Zero area box skipped: {}.".format(box_part))
                continue

            b = [int(label), float(score) * weights[t], x1, y1, x2, y2]
            if label not in new_boxes:
                new_boxes[label] = []
            new_boxes[label].append(b)

    # Sort each list in dict by score and transform it to numpy array
    for k in new_boxes:
        current_boxes = np.array(new_boxes[k])
        new_boxes[k] = current_boxes[current_boxes[:, 1].argsort()[::-1]]

    return new_boxes


def get_weighted_box(boxes, conf_type='avg'):
    """
    Create weighted box for set of boxes
    :param boxes: set of boxes to fuse
    :param conf_type: type of confidence one of 'avg' or 'max'
    :return: weighted box
    """

    box = np.zeros(6, dtype=np.float32)
    conf = 0
    conf_list = []
    for b in boxes:
        box[2:] += (b[1] * b[2:])
        conf += b[1]
        conf_list.append(b[1])
    box[0] = boxes[0][0]
    if conf_type == 'avg':
        box[1] = conf / len(boxes)
    elif conf_type == 'max':
        box[1] = np.array(conf_list).max()
    box[2:] /= conf
    return box


def find_matching_box(boxes_list, new_box, match_iou):
    best_iou = match_iou
    best_index = -1
    for i in range(len(boxes_list)):
        box = boxes_list[i]
        if box[0] != new_box[0]:
            continue
        iou = bb_intersection_over_union(box[2:], new_box[2:])
        if iou > best_iou:
            best_index = i
            best_iou = iou

    return best_index, best_iou


def weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=0.55, skip_box_thr=0.0, conf_type='avg', allows_overflow=False):
    '''
    :param boxes_list: list of boxes predictions from each model, each box is 4 numbers.
    It has 3 dimensions (models_number, model_preds, 4)
    Order of boxes: x1, y1, x2, y2. We expect float normalized coordinates
    :param scores_list: list of scores for each model
    :param labels_list: list of labels for each model
    :param weights: list of weights for each model. Default: None, which means weight == 1 for each model
    :param iou_thr: IoU value for boxes to be a match
    :param skip_box_thr: exclude boxes with score lower than this variable
    :param conf_type: how to calculate confidence in weighted boxes. 'avg': average value, 'max': maximum value
    :param allows_overflow: false if we want confidence score not exceed 1.0
    :return: boxes: boxes coordinates (Order of boxes: x1, y1, x2, y2).
    :return: scores: confidence scores
    :return: labels: boxes labels
    '''

    if weights is None:
        weights = np.ones(len(boxes_list))
    if len(weights) != len(boxes_list):
        print('Warning: incorrect number of weights {}. Must be: {}. Set weights equal to 1.'.format(len(weights), len(boxes_list)))
        weights = np.ones(len(boxes_list))
    weights = np.array(weights)

    if conf_type not in ['avg', 'max']:
        print('Unknown conf_type: {}. Must be "avg" or "max"'.format(conf_type))
        exit()

    filtered_boxes = prefilter_boxes(boxes_list, scores_list, labels_list, weights, skip_box_thr)
    if len(filtered_boxes) == 0:
        return np.zeros((0, 4)), np.zeros((0,)), np.zeros((0,))

    overall_boxes = []
    for label in filtered_boxes:
        boxes = filtered_boxes[label]
        new_boxes = []
        weighted_boxes = []

        # Clusterize boxes
        for j in range(0, len(boxes)):
            index, best_iou = find_matching_box(weighted_boxes, boxes[j], iou_thr)
            if index != -1:
                new_boxes[index].append(boxes[j])
                weighted_boxes[index] = get_weighted_box(new_boxes[index], conf_type)
            else:
                new_boxes.append([boxes[j].copy()])
                weighted_boxes.append(boxes[j].copy())

        # Rescale confidence based on number of models and boxes
        for i in range(len(new_boxes)):
            if not allows_overflow:
                weighted_boxes[i][1] = weighted_boxes[i][1] * min(weights.sum(), len(new_boxes[i])) / weights.sum()
            else:
                weighted_boxes[i][1] = weighted_boxes[i][1] * len(new_boxes[i]) / weights.sum()
        overall_boxes.append(np.array(weighted_boxes))

    overall_boxes = np.concatenate(overall_boxes, axis=0)
    overall_boxes = overall_boxes[overall_boxes[:, 1].argsort()[::-1]]
    boxes = overall_boxes[:, 2:]
    scores = overall_boxes[:, 1]
    labels = overall_boxes[:, 0]
    return boxes, scores, labels

In [ ]:
# fusion validation results
new_results = []
for image_index in range(len(dict_predictions[model_name])):
  print(image_index)
  unique_image_ids = np.unique([dict_predictions[model_name][image_index]["image_id"] for model_name in dict_models.keys()])
  assert len(unique_image_ids) == 1
  image_id = unique_image_ids[0]
  im = cv2.imread("./val2017/{0:012}.jpg".format(image_id))
  img_h, img_w = im.shape[:2]
  boxes_list, scores_list, labels_list = [], [], []
  for model_name in dict_models.keys():
    instances = dict_predictions[model_name][image_index]["instances"]
    boxes = []
    for instance in instances: # (x0, y0, w, h) -> (x0, y0, x1, y1)
      x0,y0,w,h = instance["bbox"]
      boxes.append([x0/img_w, y0/img_h, (x0+w)/img_w, (y0+h)/img_h])
    scores = [instance["score"] for instance in instances]
    labels = [instance["category_id"] for instance in instances]
    boxes_list.append(boxes)
    scores_list.append(scores)
    labels_list.append(labels)
    
    # from detectron2.structures.instances import Instances
    # im = cv2.imread("./val2017/000000000139.jpg")
    # dict_for_plot = Instances(im.shape[:2], pred_boxes=boxes, pred_classes=labels, scores=scores)
    # v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(dict_cfg["r101_c4"].DATASETS.TEST[0]), scale=1.2)
    # out = v.draw_instance_predictions(dict_for_plot)
    # cv2_imshow(out.get_image()[:, :, ::-1])

  boxes_wbf, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list)
  boxes = []
  for box in boxes_wbf: # (x0, y0, x1, y1) -> (x0, y0, w, h)
    x0, y0, x1, y1 = box 
    boxes.append([x0*img_w, y0*img_h, (x1-x0)*img_w, (y1-y0)*img_h])
  instances = []
  for instance_index in range(len(boxes)):
    bbox = boxes[instance_index]
    category_id = labels[instance_index]
    label = labels[instance_index]
    score = scores[instance_index]
    instances.append({'image_id': image_id, 'category_id':category_id, 'bbox':bbox, 'score':score, 'segmentation':[[]], 'area':bbox[-2]*bbox[-1], 'id':instance_index+1})
  new_results.append({"image_id":image_id, "instances":instances})

  # from detectron2.structures.instances import Instances
  # im = cv2.imread("./val2017/000000000139.jpg")
  # dict_for_plot = Instances(im.shape[:2], pred_boxes=[instance["bbox"] for instance in instances], pred_classes=[instance["category_id"] for instance in instances], scores=[instance["score"] for instance in instances])
  # v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(dict_cfg["r101_c4"].DATASETS.TEST[0]), scale=1.2)
  # out = v.draw_instance_predictions(dict_for_plot)
  # cv2_imshow(out.get_image()[:, :, ::-1])
  
# save to file
from detectron2.utils.file_io import PathManager
file_path = os.path.join("ensemble_instances_predictions.pth")
with PathManager.open(file_path, "wb") as f:
    torch.save(new_results, f)


In [ ]:
!cp ensemble_instances_predictions.pth gdrive/MyDrive

In [94]:
# measure individual model's performance
for dataset in ["val2017"]:
  for model_name, model in dict_models.items():
    evaluator = COCOEvaluator(dataset, dict_cfg[model_name], False)
    predictions = torch.load("ensemble_instances_predictions.pth")
    evaluator._predictions = predictions
    evaluator.evaluate()

WARNING [01/24 11:45:33 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass tasks in directly
[01/24 11:45:39 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[01/24 11:45:39 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.31s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
COCOeval_opt.evaluate() finished in 9.08 seconds.
Accumulating evaluation results...
COCOeval_opt.accumulate() finished in 1.29 seconds.
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.459
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.665
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.294
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium 